In [3]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
.appName("ShobhitApp") \
.getOrCreate()

In [4]:
from pyspark.sql.functions import col,explode

In [9]:
%%sh
hdfs dfs -cat /user/itv000197/multRowwithArray.json

[ 
    {
   "USERID":1,
   "Name":"John",
   "Mobile":82345678,
   "Flag":true,
   "Some_array":[1,2,3,4,5]
    },
     {
   "USERID":2,
   "Name":"Sam",
   "Mobile":72345678,
   "Flag":false,
   "Some_array":[11,12,3,14,5]
    }

]

In [10]:
df = spark.read.option("multiline","true").json("/user/itv000197/multRowwithArray.json")

In [11]:
#df.schema.json()
df.schema.simpleString()

'struct<Flag:boolean,Mobile:bigint,Name:string,Some_array:array<bigint>,USERID:bigint>'

In [12]:
df.printSchema()

root
 |-- Flag: boolean (nullable = true)
 |-- Mobile: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Some_array: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- USERID: long (nullable = true)



In [13]:
df.show()

+-----+--------+----+------------------+------+
| Flag|  Mobile|Name|        Some_array|USERID|
+-----+--------+----+------------------+------+
| true|82345678|John|   [1, 2, 3, 4, 5]|     1|
|false|72345678| Sam|[11, 12, 3, 14, 5]|     2|
+-----+--------+----+------------------+------+



We can now explode the array

In [14]:
df.select("Flag","Mobile","Name","USERID",explode(col("Some_array")).alias("data_col"))

Flag,Mobile,Name,USERID,data_col
true,82345678,John,1,1
true,82345678,John,1,2
true,82345678,John,1,3
true,82345678,John,1,4
true,82345678,John,1,5
false,72345678,Sam,2,11
false,72345678,Sam,2,12
false,72345678,Sam,2,3
false,72345678,Sam,2,14
false,72345678,Sam,2,5
